In [47]:
import os
import json
from typing import Dict, List, Optional
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field, validator

In [48]:

load_dotenv()

True

In [ ]:
%pip install -qU "langchain[groq]"

In [43]:
from langchain.chat_models import init_chat_model

In [56]:
llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [33]:
from typing import List, TypedDict, Annotated

In [104]:
class State(TypedDict):
    question: str
    decision: str
    

In [49]:
from typing import Optional

from typing_extensions import Annotated, TypedDict
from pydantic import BaseModel, Field


class LeadDetails(BaseModel):
    "Complete details of a lead in a CRM system."
    salutation: Optional[str] = Field(default='null', pattern="^(Mr|Mrs|Ms|Dr)$", description="Salutation of the lead.")
    first_name: str = Field(..., min_length=2, max_length=50, description="First name of the lead.")
    last_name: str = Field(..., min_length=2, max_length=50, description="Last name of the lead.")
    company: str = Field(..., max_length=100, description="Company name of the lead.")
    email: str = Field(..., pattern=r"^\S+@\S+\.\S+$", description="Email address of the lead.")
    gender: Optional[str] = Field(default='null', pattern="^(Male|Female|Other)$")
    lead_source: Optional[str] = Field(default='null', description="Lead source of the lead.")
    lead_score: Optional[int] = Field(default=0, ge=0, le=100, description="Lead score of the lead.")
    lead_value: Optional[str] = Field(default='null', ge=0, description="Lead value of the lead.")
    website: Optional[str] = Field(default='null', description="Website of the lead.")
    status: str = Field(default='new', pattern="^(new|contacted|qualified|lost)$", description="Status of the lead.")
    converted: Optional[int] = Field(default=0, description="Converted status of the lead.")
    revenue: Optional[int] = Field(default=0, ge=0, description="Revenue of the lead.")
    company_constitution: Optional[str] = Field(default='null', description="Company constitution of the lead.")
    company_incorporation_date: Optional[str] = Field(default='null', description="Company incorporation date of the lead.")
    mobile_code: Optional[str] = Field(default='null', description="Mobile code of the lead.")
    mobile_no: Optional[str] = Field(default='null', min_length=10, max_length=15, description="Mobile number of the lead.")
    employees: Optional[int] = Field(default=0, ge=0, description="Number of employees of the lead.")
    territory_id: Optional[str] = Field(default='null', description="Territory ID of the lead.")
    industry_id: Optional[str] = Field(default='null', description="Industry ID of the lead.")
    department_id: Optional[str] = Field(default='null', description="Department ID of the lead.")
    
       
    

In [50]:
#test
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI


# llm = ChatOpenAI(model="gpt-4o", temperature=0)

parser = JsonOutputParser(pydantic_object=LeadDetails)

In [10]:
def llm_parser():
    
    parser = JsonOutputParser(pydantic_object=LeadDetails)
    llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")
    prompt = ChatPromptTemplate([
    ("system","""You Are a CRM system details: extraxter agent you need to collect the following details from the given user query.\n\n
    1. First Name: 
    2. Last Name: 
    3. Company: 
    4. Email:
    so collect the above details from the user query and return it.
    
    STRICTLY FOLLOW THE INSTRUCTIONS:
    parse the available and given instructions only dont replace with anything for other input.
    
    Answer the user query. Wrap the output in `json` tags\n{format_instructions}
    \n"""),
    ("user", "this is the user input:{query}")]).partial(format_instructions=parser.get_format_instructions())
    
    
    chain = prompt | llm | parser
    query = "create a lead jhon doe from antar crm with antar@gmail.com and the lead source is hentai the lead score is 7"
    result = chain.invoke({"query": query})
    return result
    

In [51]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("system","""You Are a CRM system details: extraxter agent you need to collect the following details from the given user query.\n\n
    1. First Name: 
    2. Last Name: 
    3. Company: 
    4. Email:
    so collect the above details from the user query and return it.
    
    STRICTLY FOLLOW THE INSTRUCTIONS:
    parse the available data from the USER QUERY and give the structued result if result not found 
    DONT REPLACE WITH NONE. You can Replace with 'null'.
    
    Answer the user query. Wrap the output in `json` tags\n{format_instructions}
    \n"""),
    ("user", "this is the user input:{query}")]).partial(format_instructions=parser.get_format_instructions())

In [16]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Complete details of a lead in a CRM system.", "properties": {"salutation": {"anyOf": [{"pattern": "^(Mr|Mrs|Ms|Dr)$", "type": "string"}, {"type": "null"}], "default": "null", "description": "Salutation of the lead.", "title": "Salutation"}, "first_name": {"description": "First name of the lead.", "maxLength": 50, "minLength": 2, "title": "First Name", "type": "string"}, "last_name": {"description": "Last name of the lead.", "maxLength": 50, "minLength": 2, "title": "Last Name", "type": "string"}, "company": {"descripti

In [53]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = os.getenv("AZURE_INFERENCE_SDK_ENDPOINT", "your_endpoint_here")
model_name = os.getenv("DEPLOYMENT_NAME", "Meta-Llama-3-8B-Instruct")
key = os.getenv("AZURE_INFERENCE_SDK_KEY", "YOUR_KEY_HERE")
client = ChatCompletionsClient(endpoint=endpoint, credential=AzureKeyCredential(key))


In [22]:
%pip install -U langchain-azure-ai

  Using cached numpy-1.26.4-cp313-cp313-macosx_15_0_arm64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.3/949.3 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.2
    Uninstalling numpy-2.2.2:
      Successfully uninstalled numpy-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qdrant-client 1.13.2 requires numpy>=2.1.0; python_version >= "3.13", but you have numpy 1.26.4 which is incompatible.
browser-use 0.1.26 requires langchain==0.3.14, but you have langchain 0.3.19 which is incompatible.
browser-use 0.1.26 requires langchain-ollama==0.2.2, but you have langchain-ollama 0.2.3 which is incompatible.
transformers 4.48.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [58]:
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

llm = AzureAIChatCompletionsModel(
    model_name=model_name,
    credential=AzureKeyCredential(key),
    endpoint=endpoint,
    temperature=0.1,
    
)

llm.invoke('Tell me a joke and include some emojis')

AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself? 🚴\u200d♂️\n\nBecause it was two-tired! 😴👀\n\nHope that made you laugh! 😄", additional_kwargs={}, response_metadata={'model': 'Meta-Llama-3-8B-Instruct', 'token_usage': {'input_tokens': 18, 'output_tokens': 44, 'total_tokens': 62}, 'finish_reason': 'stop'}, id='run-114bd73b-748f-4b5b-be73-b0302274dc39-0', usage_metadata={'input_tokens': 18, 'output_tokens': 44, 'total_tokens': 62})

In [62]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

chat_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    openai_api_key=OPENAI_API_KEY,
)

In [64]:
chain = prompt | llm | parser
query = "create a lead jhon"
result = chain.invoke({"query": query})
result

{'salutation': None,
 'first_name': 'Jhon',
 'last_name': None,
 'company': 'create a lead',
 'email': None,
 'gender': None,
 'lead_source': None,
 'lead_score': 0,
 'lead_value': None,
 'website': None,
 'status': 'new',
 'converted': 0,
 'revenue': 0,
 'company_constitution': None,
 'company_incorporation_date': None,
 'mobile_code': None,
 'mobile_no': None,
 'employees': 0,
 'territory_id': None,
 'industry_id': None,
 'department_id': None}

In [25]:
colleceted_data: dict = {}
colleceted_data.update(result)

In [26]:
final_LeadData = LeadDetails(**colleceted_data).model_dump()

In [27]:
final_LeadData

{'salutation': None,
 'first_name': 'Jhon',
 'last_name': 'Doe',
 'company': 'Extraxter',
 'email': 'jhon.doe@extraxter.com',
 'gender': None,
 'lead_source': None,
 'lead_score': 0,
 'lead_value': None,
 'website': None,
 'status': 'new',
 'converted': 0,
 'revenue': 0,
 'company_constitution': None,
 'company_incorporation_date': None,
 'mobile_code': None,
 'mobile_no': None,
 'employees': 0,
 'territory_id': None,
 'industry_id': None,
 'department_id': None}

In [217]:
def missing_fields(colleceted_data, required_fields):
    missing_fields = []
    for field in required_fields:
        if field not in colleceted_data:
            missing_fields.append(field)
    return missing_fields
    

_IncompleteInputError: incomplete input (237004170.py, line 1)

In [ ]:
def get_input():
    MANDATORY_FIELDS = ["first_name", "last_name", "company", "email"]
    colleceted_data: dict = {}
    
    while True:
        missing_fields = missing_fields(colleceted_data, MANDATORY_FIELDS)
        if missing_fields:
            query = input(f"Enter the required Data {','.join(missing_fields)}: ")
            result = chain.invoke({"query": query})
            colleceted_data.update(result)
        if not missing_fields:
            if input("Do you want to add more details? (y/n): ").lower() == "n":
                break
            else:
                query = input("Enter the Data: ")
                result = chain.invoke({"query": query})
                colleceted_data.update(result)
    return colleceted_data

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


def decide_intent(state: State):
    
    "Decide the operation to be performed based on the user query."
    
    system_message = """You are a CRM System Manager for Antar CRM.
    Here There are several operations CREATE, READ, UPDATE and DELETE.
    User input will pass through the system and you need to decide which operation to perform.
    
    FOR EXAMPLE:
      If the user says create lead or any other querys related to the creation of operations then you need to perform the create operation.
      likewise you need to do perfrom all the other four operations are create read update and delete.
      
    YOUR TASK:
        1. Extract the operation from the user query.
        2. Return the operation name.
        if related to create then it return with 'create' and so on.
        if the task is not related to any of the above operations then return 'unkonwn'.
      """
      
    prompt = ChatPromptTemplate([
        ("system", system_message),
        ("user", "this is the user input:{query}")])
    chain = prompt | llm
    response = chain.invoke({"query": state['query']})
    response = response.content.strip().lower()
    if response != 'unknown':
        state['decision'] = response
    else:
        state['decision'] = 'unknown'
    return state
    
    

In [1]:
%pip install azure.identity azure-ai-inference

  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
Using cached PyJWT-2.10.1-py3-none-any.whl (22 kB)
Note: you may need to restart the kernel to use updated packages.


In [60]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = os.getenv("AZURE_INFERENCE_SDK_ENDPOINT", "https://azurefoundry4137253792.services.ai.azure.com/models")
model_name = os.getenv("DEPLOYMENT_NAME", "Meta-Llama-3-8B-Instruct")
key = os.getenv("AZURE_INFERENCE_SDK_KEY", "YOUR_KEY_HERE")
client = ChatCompletionsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

response = client.complete(
  messages=[
    SystemMessage(content="You are a helpful assistant."),
    UserMessage(content="What are 3 things to visit in Seattle?")
  ],
  model = model_name,
  max_tokens=1000
)

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Seattle! Such a great city! Here are three must-visit attractions in Seattle:\n\n1. **The Space Needle**: This iconic 605-foot tall tower is a symbol of Seattle and offers breathtaking 360-degree views of the city, Puget Sound, and the surrounding mountains. You can take a elevator ride to the top for panoramic views and learn about the history of the tower through interactive exhibits.\n2. **Pike Place Market**: This historic farmers market is a must-visit for foodies and shoppers alike. You can sample local foods, watch fishmongers in action, and browse through stalls selling fresh produce, crafts, and souvenirs. Be sure to check out the original Starbucks store, which is located nearby.\n3. **The Chihuly Garden and Glass**: This unique attraction showcases the stunning glass artwork of Dale Chihuly in a beautiful garden setting. The exhibit features hundreds of Chihuly's pi

In [ ]:
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint